In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from lib_processing import data_processing as processing
import nltk
from transformers import AutoTokenizer
import torch
from tqdm import tqdm
import re
import spacy

nltk.download('stopwords')
from nltk.corpus import stopwords
import multiprocessing

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\lukas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
corpus_df = processing.get_data()

Retrieving data from ParlaMint-AT...
./data/parlamint_at_extracted\README-AT.md
./data/parlamint_at_extracted\ParlaMint-AT.TEI\00README.txt
./data/parlamint_at_extracted\ParlaMint-AT.TEI\ParlaMint-AT-listOrg.xml
./data/parlamint_at_extracted\ParlaMint-AT.TEI\ParlaMint-AT-listPerson.xml
./data/parlamint_at_extracted\ParlaMint-AT.TEI\ParlaMint-AT.xml
./data/parlamint_at_extracted\ParlaMint-AT.TEI\ParlaMint-taxonomy-CHES.xml
./data/parlamint_at_extracted\ParlaMint-AT.TEI\ParlaMint-taxonomy-parla.legislature.xml
./data/parlamint_at_extracted\ParlaMint-AT.TEI\ParlaMint-taxonomy-politicalOrientation.xml
./data/parlamint_at_extracted\ParlaMint-AT.TEI\ParlaMint-taxonomy-speaker_types.xml
./data/parlamint_at_extracted\ParlaMint-AT.TEI\ParlaMint-taxonomy-subcorpus.xml
./data/parlamint_at_extracted\ParlaMint-AT.TEI\1996\ParlaMint-AT_1996-01-15-020-XX-NRSITZ-00001.xml
./data/parlamint_at_extracted\ParlaMint-AT.TEI\1996\ParlaMint-AT_1996-01-15-020-XX-NRSITZ-00002.xml
./data/parlamint_at_extracted\P

In [4]:
corpus_df.head()

,ID,text,Text_ID,Title,Date,Body,Term,Session,Meeting,Sitting,...,Speaker_MP,Speaker_minister,Speaker_party,Speaker_party_name,Party_status,Party_orientation,Speaker_ID,Speaker_name,Speaker_gender,Speaker_birth
0,ParlaMint-AT_1996-01-15-020-XX-NRSITZ-00001_d7...,Bitte Platz zu nehmen. – Ich bitte auch die Ph...,ParlaMint-AT_1996-01-15-020-XX-NRSITZ-00001,"Sitting Number 1, Legislative period XX, Monda...",1996-01-15,Lower house,Legislative period XX,-,-,Sitting 1,...,MP,notMinister,SPÖ,parliamentary group of the Social Democratic P...,Coalition,Centre-left,PAD_00334,"Fischer, Heinz",M,1938
1,ParlaMint-AT_1996-01-15-020-XX-NRSITZ-00001_d7...,Der Herr Bundespräsident hat mit Entschließung...,ParlaMint-AT_1996-01-15-020-XX-NRSITZ-00001,"Sitting Number 1, Legislative period XX, Monda...",1996-01-15,Lower house,Legislative period XX,-,-,Sitting 1,...,MP,notMinister,SPÖ,parliamentary group of the Social Democratic P...,Coalition,Centre-left,PAD_00334,"Fischer, Heinz",M,1938
2,ParlaMint-AT_1996-01-15-020-XX-NRSITZ-00001_d7...,Wir gehen in die Tagesordnung ein und gelangen...,ParlaMint-AT_1996-01-15-020-XX-NRSITZ-00001,"Sitting Number 1, Legislative period XX, Monda...",1996-01-15,Lower house,Legislative period XX,-,-,Sitting 1,...,MP,notMinister,SPÖ,parliamentary group of the Social Democratic P...,Coalition,Centre-left,PAD_00334,"Fischer, Heinz",M,1938
3,ParlaMint-AT_1996-01-15-020-XX-NRSITZ-00001_d7...,Sie werden geloben unverbrüchliche Treue der R...,ParlaMint-AT_1996-01-15-020-XX-NRSITZ-00001,"Sitting Number 1, Legislative period XX, Monda...",1996-01-15,Lower house,Legislative period XX,-,-,Sitting 1,...,MP,notMinister,SPÖ,parliamentary group of the Social Democratic P...,Coalition,Centre-left,PAD_01153,"Parfuss, Ludmilla",F,1942
4,ParlaMint-AT_1996-01-15-020-XX-NRSITZ-00001_d7...,Danke. Die Angelobung ist damit beendet. Meine...,ParlaMint-AT_1996-01-15-020-XX-NRSITZ-00001,"Sitting Number 1, Legislative period XX, Monda...",1996-01-15,Lower house,Legislative period XX,-,-,Sitting 1,...,MP,notMinister,SPÖ,parliamentary group of the Social Democratic P...,Coalition,Centre-left,PAD_00334,"Fischer, Heinz",M,1938


converting date and extracting month and year to be albe to filter and generate a smaller POC data set.

In [9]:
df_sent = pd.read_csv("data/corpus_normalised_tokens.csv")
# Convert to datetime
df_sent['Date'] = pd.to_datetime(df_sent['Date'])

# Extract year and month
df_sent['year'] = df_sent['Date'].dt.year
df_sent['month'] = df_sent['Date'].dt.month
df_sent.head()

C:\Users\lukas\AppData\Local\Temp\ipykernel_18308\3078247261.py:1: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sent = pd.read_csv("data/corpus_normalised_tokens.csv")


,Unnamed: 0,ID,text,Text_ID,Title,Date,Body,Term,Session,Meeting,...,Speaker_party,Speaker_party_name,Party_status,Party_orientation,Speaker_ID,Speaker_name,Speaker_gender,Speaker_birth,year,month
0,0,ParlaMint-AT_1996-01-15-020-XX-NRSITZ-00001_d7...,"platz zu. ich auch die photographen, ihre täti...",ParlaMint-AT_1996-01-15-020-XX-NRSITZ-00001,"Sitting Number 1, Legislative period XX, Monda...",1996-01-15,Lower house,Legislative period XX,-,-,...,SPÖ,parliamentary group of the Social Democratic P...,Coalition,Centre-left,PAD_00334,"Fischer, Heinz",M,1938,1996,1
1,1,ParlaMint-AT_1996-01-15-020-XX-NRSITZ-00001_d7...,der hat mit entschließung vom 9. jänner gemäß ...,ParlaMint-AT_1996-01-15-020-XX-NRSITZ-00001,"Sitting Number 1, Legislative period XX, Monda...",1996-01-15,Lower house,Legislative period XX,-,-,...,SPÖ,parliamentary group of the Social Democratic P...,Coalition,Centre-left,PAD_00334,"Fischer, Heinz",M,1938,1996,1
2,2,ParlaMint-AT_1996-01-15-020-XX-NRSITZ-00001_d7...,wir in die tagesordnung ein und gelangen zum 1...,ParlaMint-AT_1996-01-15-020-XX-NRSITZ-00001,"Sitting Number 1, Legislative period XX, Monda...",1996-01-15,Lower house,Legislative period XX,-,-,...,SPÖ,parliamentary group of the Social Democratic P...,Coalition,Centre-left,PAD_00334,"Fischer, Heinz",M,1938,1996,1
3,3,ParlaMint-AT_1996-01-15-020-XX-NRSITZ-00001_d7...,sie geloben unverbrüchliche treue der republik...,ParlaMint-AT_1996-01-15-020-XX-NRSITZ-00001,"Sitting Number 1, Legislative period XX, Monda...",1996-01-15,Lower house,Legislative period XX,-,-,...,SPÖ,parliamentary group of the Social Democratic P...,Coalition,Centre-left,PAD_01153,"Parfuss, Ludmilla",F,1942,1996,1
4,4,ParlaMint-AT_1996-01-15-020-XX-NRSITZ-00001_d7...,. die angelobung ist damit beendet.! sie sich ...,ParlaMint-AT_1996-01-15-020-XX-NRSITZ-00001,"Sitting Number 1, Legislative period XX, Monda...",1996-01-15,Lower house,Legislative period XX,-,-,...,SPÖ,parliamentary group of the Social Democratic P...,Coalition,Centre-left,PAD_00334,"Fischer, Heinz",M,1938,1996,1


In [10]:
df_sent["text"].iloc[0]

'platz zu. ich auch die photographen, ihre tätigkeit einzustellen. die erste und konstituierende sitzung des nationalrates der xx. gesetzgebungsperiode des nationalrates eröffnen. ich begrüße hiezu alle mitglieder des hohen hauses herzlich, und zwar sowohl jene, die neuerlich in den nationalrat gewählt wurden, als auch jene, die erstmals an einer sitzung des nationalrates teil. mit respekt begrüße ich den herrn bundespräsidenten, der bei dieser sitzung anwesend ist. auch jene bürgerinnen und bürger begrüßen, die als zuseher die liveübertragung dieser sitzung im fern verfolgen. einberufung der ordentlichen tagung 1996'

In [11]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
from germansentiment import SentimentModel

def extract_compound_sentiment_from_speech(speech, model,console_out):
    all_probs = []
    sentences = sent_tokenize(speech, language='german')
    print(len(sentences))
    sentiment_weight = {'positive': 1, 'negative': -1, 'neutral': 0}
    compound_scores = []

    for sentence in sentences:
        _, result = model.predict_sentiment([sentence], output_probabilities=True)
        prob_pairs = result[0]
        prob_dict = {label: prob for label, prob in prob_pairs}
        all_probs.append(prob_dict)

        # Sentence-level compound score
        score = sum(prob_dict[label] * sentiment_weight[label] for label in sentiment_weight)
        compound_scores.append(score)

    # Overall compound sentiment and variance
    compound_score = np.mean(compound_scores)
    compound_variance = np.var(compound_scores)

    if console_out == True:
        print("Compound sentiment score:", round(compound_score, 4))
        print("Compound sentiment variance:", round(compound_variance, 4))

    return compound_score, compound_variance,



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\lukas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\lukas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [12]:

model = SentimentModel()

compound_score, compound_variance = extract_compound_sentiment_from_speech(df_sent["text"].iloc[0], model,console_out=True)
df_sent["text"].iloc[0]

8
Compound sentiment score: -0.1116
Compound sentiment variance: 0.0879


'platz zu. ich auch die photographen, ihre tätigkeit einzustellen. die erste und konstituierende sitzung des nationalrates der xx. gesetzgebungsperiode des nationalrates eröffnen. ich begrüße hiezu alle mitglieder des hohen hauses herzlich, und zwar sowohl jene, die neuerlich in den nationalrat gewählt wurden, als auch jene, die erstmals an einer sitzung des nationalrates teil. mit respekt begrüße ich den herrn bundespräsidenten, der bei dieser sitzung anwesend ist. auch jene bürgerinnen und bürger begrüßen, die als zuseher die liveübertragung dieser sitzung im fern verfolgen. einberufung der ordentlichen tagung 1996'

In [14]:
df_speeches = df_sent[["text"]].copy()

In [15]:
df_speeches.head()

,text
0,"platz zu. ich auch die photographen, ihre täti..."
1,der hat mit entschließung vom 9. jänner gemäß ...
2,wir in die tagesordnung ein und gelangen zum 1...
3,sie geloben unverbrüchliche treue der republik...
4,. die angelobung ist damit beendet.! sie sich ...


In [48]:
texts = df_speeches['text'].astype(str).fillna('').str.strip().tolist()

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
from tqdm import tqdm

# Load tokenizer and model
model_name = "oliverguhr/german-sentiment-bert"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()
print("Model is on:", next(model.parameters()).device)

texts = df_speeches['text']

batch_size = 32
compound_scores = []
compound_variances = []
df_sentiment_result = pd.DataFrame()

sentiment_weight = {'positive': 1, 'negative': -1, 'neutral': 0}
labels = ['positive','negative', 'neutral', ]  # label order in model output

def sentence_level_scores(sentence):
    # Tokenize sentence, move to device
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        probs = torch.softmax(outputs.logits, dim=-1).cpu().numpy()[0]
    prob_dict = dict(zip(labels, probs))
    # Compound score for this sentence
    score = sum(prob_dict[label] * sentiment_weight[label] for label in sentiment_weight)
    return prob_dict, score

from nltk.tokenize import sent_tokenize

for i in tqdm(range(0, len(texts), batch_size), desc="Processing batches"):
    batch = texts[i:i+batch_size]
    for speech in batch:
        sentences = sent_tokenize(speech, language='german')
        all_probs = []
        compound_scores_sentences = []

        for sent in sentences:
            prob_dict, score = sentence_level_scores(sent)
            all_probs.append(prob_dict)
            compound_scores_sentences.append(score)

        # Aggregate over sentences
        avg_probs = {label: np.mean([p[label] for p in all_probs]) for label in labels}
        var_probs = {label: np.var([p[label] for p in all_probs]) for label in labels}

        compound_score = np.mean(compound_scores_sentences)
        compound_variance = np.var(compound_scores_sentences)

        compound_scores.append(compound_score)
        compound_variances.append(compound_variance)

# Add to DataFrame
df_sentiment_result['compound_score'] = compound_scores
df_sentiment_result['compound_variance'] = compound_variances
df_sentiment_result.to_csv("data/extractted_sentiment")

Model is on: cuda:0


Processing batches:   0%|          | 13/7243 [00:33<5:14:07,  2.61s/it]


KeyboardInterrupt: 

In [59]:
df_sentiment_result.head()

""


In [63]:
import torch
print(torch.cuda.is_available())           # Should be True
print(torch.cuda.get_device_name(0))

False


AssertionError: Torch not compiled with CUDA enabled

In [5]:
import torch
print(torch.cuda.is_available())  # Should be True
print(torch.version.cuda)   

True
11.8
